In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pylab as plt

import seaborn as sns

from skspatial.objects import Line, Plane
from skspatial.plotting import plot_3d


from skspatial.objects import Line, Cylinder, Point, Points
from skspatial.plotting import plot_3d

import phasespace

import tensorflow

import bisect
import numpy as np
import matplotlib.pylab as plt
import pandas as pd

import seaborn as sns

import numpy as np
from sklearn.mixture import GaussianMixture
from scipy.stats import multivariate_normal

import numpy as np
from scipy.interpolate import griddata
from scipy.integrate import quad, trapezoid
from scipy.interpolate import CubicSpline

import matplotlib.pylab as plt
from scipy import stats
from matplotlib import cm
from matplotlib.ticker import LinearLocator
import matplotlib

from scipy.interpolate import LinearNDInterpolator
import scipy

from scipy import integrate 




import dm_generation_tools as dgt
import detector_simulation_tools as dst


import time

####################################
import warnings
# Suppress all warnings
warnings.filterwarnings("ignore")


import pickle

In [ ]:
print(f'{np.__version__ = }')
print(f'{matplotlib.__version__ = }')
print(f'{pd.__version__ = }')
print(f'{scipy.__version__ = }')

In [ ]:
# We need this file. Make sure there is a soft link to it
infilename_for_eloss = 'muons_summary_from_GEANT4_simulations.parquet'
df_eloss = pd.read_parquet(infilename_for_eloss)
df_eloss

In [ ]:
# Make some plots
e_initials = df_eloss['e_initial'].unique()
e_initials


plt.figure(figsize=(12,4))

for idx,ei in enumerate(e_initials[0:2]):
    print(f"ei: {ei}")
    filter = df_eloss['e_initial']==ei

    z = df_eloss[filter]['z']
    ef = ei - df_eloss[filter]['e']

    print(len(z))
    print(len(ef))

    plt.subplot(1,2,idx+1)
    plt.plot(z,ef,'.',markersize=0.1)
    plt.xlabel('z (m)')
    plt.ylabel('Final energy (GeV)')
    plt.title(f'Initial energy: {ei} GeV')

In [ ]:
def find_neighbors(sorted_list, x):
    """
    Given sorted_list (ascending) and a value x,
    return (low, high) where:
      - low  = the largest element <= x (or None if x < sorted_list[0])
      - high = the smallest element >= x (or None if x > sorted_list[-1])
    """
    idx = bisect.bisect_left(sorted_list, x)
    # idx is the insertion point to keep the list sorted.
    if idx == 0:
        # x is <= first element
        return None, sorted_list[0]
    elif idx == len(sorted_list):
        # x is greater than all elements
        return sorted_list[-1], None
    else:
        # sorted_list[idx-1] < x <= sorted_list[idx]
        return sorted_list[idx-1], sorted_list[idx]

#####################################################################################

def energy_after_traveling_distance(e_initials, zvals, eivals, efvals, E_muon, distance, make_plots=False, ngendata=1, verbose=False):
    
    start = time.time()

    # My own 
    d = distance
    dwidth = 0.01 * d

    #print(e_initials)
    #E_muon = 45040
    elo,ehi = find_neighbors(e_initials, E_muon)
    delta_e = ehi - elo

    if verbose:
        print(f"In function: time to run A: {time.time() - start:.2f} seconds")

    #print(elo, ehi, delta_e)
    
    frequencies = []
    eis = []
    efs = []
    
    #plt.figure(figsize=(16,8))
    
    # First find the ranges
    elo_min = [1e99, 1e99]
    ehi_max = [-1e99, -1e99]
    
    #filter_distance = (zvals > d-dwidth) & (zvals < d+dwidth)
    
    for i,Ei in enumerate([elo,ehi]):
        
        filter = (eivals==Ei)# & filter_distance
        #filter = filter & 
        
        #vals = Ei - efvals[filter]
        vals = efvals[filter]
    
        if len(vals) < 10:
            if verbose:
                print("NO DATA TO WORK WITH")
            return -1*np.ones(ngendata,dtype=int)
            #continue
            
        test_min = min(vals)
        test_max = max(vals)
        if test_min <= elo_min[i]:
            elo_min[i] = test_min
        if test_max >= ehi_max[i]:
            ehi_max[i] = test_max
    if verbose:
        print(f"In function: time to run B: {time.time() - start:.2f} seconds")
    
    for i,Ei in enumerate([elo,ehi]):
        #Ei = 30000
        
        filter = (eivals==Ei)# & filter_distance
        #filter = filter & (zvals > d-dwidth) & (zvals < d+dwidth)
        
        #vals = Ei - efvals[filter]
        vals = efvals[filter]

        if make_plots:
            plt.subplot(2,2,1)
            plt.hist(vals, bins=100, range=(elo_min[i], ehi_max[i]),label=f'{Ei}', alpha=0.5);
            plt.legend()

        if len(vals) < 10:
            if verbose:
                print("NO DATA TO WORK WITH")
            return -1*np.ones(ngendata,dtype=int)
            #continue

        kde = stats.gaussian_kde(vals)
        xpts = np.linspace(elo_min[i], ehi_max[i], 100)
        ypts = kde(xpts)
        #frequencies = kde.evaluate(xpts)
    
        # Shift up the lower one
        #if i==0:
        #    xpts += delta_e
    
        # Normalize the xpts
        e_range = ehi_max[i] - elo_min[i]
        xpts -= elo_min[i]
        xpts /= e_range

        if make_plots:
            plt.subplot(2,2,2)
            plt.plot(xpts,ypts, label=f'{Ei}')
            #plt.xlim(0,1.1*ehi)
            plt.legend()
    
        efs += xpts.tolist()
        eis += (Ei * np.ones_like(xpts)).tolist()
        frequencies += ypts.tolist()

    if verbose:
        print(f"In function: time to run C: {time.time() - start:.2f} seconds")

    data = np.array([eis, efs])
    #print(data.shape, len(frequencies))
    interp = LinearNDInterpolator(data.T, frequencies )

    if verbose:
        print(f"In function: time to run D: {time.time() - start:.2f} seconds")

    #xpts_temp = np.linspace(elo_min,ehi_max,100)
    xpts_temp = np.linspace(0,1,100)
    
    ypts_temp = interp(E_muon,xpts_temp);
    
    filter = ypts_temp==ypts_temp
    
    # Cut out the nans since some of the points are out of range
    xpts = xpts_temp[filter]
    ypts = ypts_temp[filter]
    
    # Shift the xpoints down
    #xpts -= (ehi - E_muon)

    if make_plots:
        #plt.figure()
        plt.plot(xpts,ypts, label=f'{E_muon}')
        plt.legend()
    
    # Sample points
    dx = xpts[1] - xpts[0]
    #print(dx)
    #print(ypts)
    
    cdf = np.cumsum(ypts)*dx
    #print(cdf)
    
    cdf /= cdf[-1]
    
    #print(cdf)
    if verbose:
        print(f"In function: time to run E: {time.time() - start:.2f} seconds")

    if make_plots:
        plt.figure(figsize=(12,4))
        plt.subplot(1,2,1)
        plt.plot(xpts, cdf, label='CDF')
        #print(cdf)
        plt.legend()
    
    #filter = (xpts>=elo_min) & (xpts<=ehi_max)
    #print(cdf)
    #print(cdf[filter])
    
    spl = CubicSpline(xpts, cdf)

    if verbose:
        print(f"In function: time to run F: {time.time() - start:.2f} seconds")

    gendata = []
    #ngendata = 5000
    nfail = 0
    
    # Scaling
    frac_of_diff = (E_muon - elo) / (ehi - elo)
    #print(f"{frac_of_diff = }")
    e_range_0 = ehi_max[0] - elo_min[0]
    e_range_1 = ehi_max[1] - elo_min[1]
    #print(f"{e_range_0 = }")
    #print(f"{e_range_1 = }")
    
    e_muon_range = e_range_0 + ((e_range_1 - e_range_0)*frac_of_diff)
    e_muon_lo   = elo_min[0] + ((elo_min[1] - elo_min[0])*frac_of_diff)
    #print(f"{e_muon_range = }")
    #print(f"{e_muon_lo = }")
    
    #ngendata = 1
    icount = 0
    
    while icount < ngendata:
        #print(icount, ngendata)
        u = np.random.random() # Generates a float between 0.0 and 1.0
    
        #print(f"In function: time to run G - a: {time.time() - start:.2f} seconds")

        ynew = spl.solve(u)
        #print(f"In function: time to run G - b: {time.time() - start:.2f} seconds")
        
        #xnew = max(ynew)
        #filter = (ynew>0) & (ynew<ehi_max)
        filter = (ynew>0) & (ynew<1)
    
        xnew = ynew[filter]
        if len(xnew) == 1:
            good_val = xnew[0]
            good_val *= e_muon_range
            good_val += e_muon_lo
        else:
            print(u, ynew, xnew)
            nfail += 1
            continue
        #print(good_val)
        #plt.subplot(1,3,2)
        #print(xnew,u)
        #plt.plot(xnew, u, 'ro', markersize=5)
        
        gendata.append(good_val)
        icount += 1

    if make_plots:
        plt.subplot(1,2,2)
        plt.hist(gendata, bins=200)#, range=(0,1));
    
    if verbose:
        print(f"In function: time to run G: {time.time() - start:.2f} seconds")
        print(f'{nfail = }')

    return gendata
    

In [ ]:
# Test the energy loss code
e_initials = df_eloss['e_initial'].unique()
zvals = df_eloss['z'].values
eivals = df_eloss['e_initial']
efvals = df_eloss['e']
    
start = time.time()
distance = 1000
distance_width = 0.01 * distance
E_muon = 45000

elo,ehi = find_neighbors(e_initials, E_muon)
filter = (eivals==elo) | (eivals==ehi)
filter = filter & (zvals>distance - distance_width) & (zvals<distance+distance_width)

print("Here")

delta_e = energy_after_traveling_distance(e_initials, zvals[filter], eivals[filter], efvals[filter], E_muon=E_muon, distance=distance, make_plots=False, ngendata=1)
print(f"Time to run: {time.time() - start:.2f} seconds")
de = delta_e[0]
e_final = E_muon - de
print(f"{E_muon:.2f}  {de:.2f}   {e_final:.2f}")


In [ ]:
# Make some plots
e_initials = df_eloss['e_initial'].unique()
e_initials


plt.figure(figsize=(12,4))

for idx,ei in enumerate(e_initials[0:2]):
    print(f"ei: {ei}")
    filter = df_eloss['e_initial']==ei

    z = df_eloss[filter]['z']
    ef = ei - df_eloss[filter]['e']

    print(len(z))
    print(len(ef))

    plt.subplot(1,2,idx+1)
    plt.plot(z,ef,'.',markersize=0.1)
    plt.xlabel('z (m)')
    plt.ylabel('Final energy (GeV)')
    plt.title(f'Initial energy: {ei} GeV')

In [ ]:
find_neighbors(e_initials, 150)


In [ ]:
e_initials = df_eloss['e_initial']
z_vals = df_eloss['z']
ef_vals = df_eloss['e']

In [ ]:
from scipy.stats import gaussian_kde
from sklearn.neighbors import KernelDensity


In [ ]:
e_initials_unique = df_eloss['e_initial'].unique()


data = {'e_i':[], 'dist':[], 'de_min':[], 'de_max':[]}

for d in [50, 75, 100, 200, 500, 1000]:
    #d = 75 # meters
    dwidth = 0.05 * d
    print(d,dwidth)
    
    for e_i in e_initials_unique:
    
        #############################################
        
        filter_e1 = (e_initials==e_i)
        filter_d1 = (z_vals>=d-dwidth) & (z_vals<=d+dwidth)
        
        de1 = df_eloss[filter_e1 & filter_d1]['e']

        de_min = 0
        de_max = 0
        if len(de1)>0:
            de_min = min(de1)
        if len(de1)>0:
            de_max = max(de1)
    
        #print(e_i, de_min, de_max)
    
        data['e_i'].append(e_i)
        data['de_min'].append(de_min)
        data['de_max'].append(de_max)
        data['dist'].append(d)
    
df_min_max = pd.DataFrame.from_dict(data)

df_min_max

In [ ]:
fig,axes = plt.subplots(1,2,figsize=(12,5))

plt.sca(axes[0])
sns.scatterplot(data=df_min_max, x='e_i', y='de_min', hue='dist')

plt.yscale('log')
plt.xscale('log')


plt.sca(axes[1])
sns.scatterplot(data=df_min_max, x='e_i', y='de_max', hue='dist')

plt.yscale('log')
plt.xscale('log')

In [ ]:
def find_de_ranges(de1, de2):
    elo_vals = np.array([min(de1), min(de2)])
    ehi_vals = np.array([max(de1), max(de2)])

    
    elo_min = min(elo_vals)
    ehi_max = max(ehi_vals)

    #print(elo_min, ehi_max)

    return elo_vals, ehi_vals, elo_min, ehi_max
################################################################



def get_data_and_kde_for_interpolation(E_muon=100, distance=50, df_eloss=None, e_initials_unique=None, dwidth_pct=0.05):

    if E_muon <= min(e_initials_unique):
        print("E_muon is too low")
        return None, None, None, None, None
    
    dwidth = 0.05 * distance
    #print(d,dwidth)

    elo,ehi = find_neighbors(e_initials_unique, E_muon)
    print(E_muon, elo, ehi)

    filter_e1 = (e_initials==elo)
    filter_e2 = (e_initials==ehi)
    filter_d1 = (z_vals>=distance-dwidth) & (z_vals<=distance+dwidth)
    
    de1 = df_eloss[filter_e1 & filter_d1]['e']
    de2 = df_eloss[filter_e2 & filter_d1]['e']

    #print(de1)
    #####################################
    
    bandwidth = 0.1
    kde1 = gaussian_kde(de1, bw_method=bandwidth)
    kde2 = gaussian_kde(de2, bw_method=bandwidth)

    elo_vals, ehi_vals, elo_min,ehi_max = find_de_ranges(de1, de2)
    
    xpts = np.linspace(elo_min,ehi_max,500)


    return de1, de2, kde1, kde2, xpts, elo, ehi

######################################################################
def morph_distributions(E_muon, kde1, kde2, xpts, elo, ehi):

    kde1_norm_vals = kde1.pdf(xpts)
    kde2_norm_vals = kde2.pdf(xpts)

    kde1_integral = integrate.trapezoid(kde1_norm_vals, xpts)
    kde2_integral = integrate.trapezoid(kde2_norm_vals, xpts)
    
    #print(kde1_integral, kde2_integral)
    
    kde1_norm_vals /= kde1_integral
    kde2_norm_vals /= kde2_integral
    
    scale_factor = (ehi - E_muon)/(ehi-elo)
    #print(f'{E_muon=}   {elo=}  {ehi=}  {scale_factor=}')
    kde3_norm_vals =  (scale_factor*kde1_norm_vals) + ((1-scale_factor)*kde2_norm_vals)

    # Cut out values that are greater than the initial energy of the muon
    filter = xpts>E_muon
    kde3_norm_vals[filter] = 0
    
    kde3_norm_integral = integrate.trapezoid(kde3_norm_vals, xpts)
    kde3_norm_vals /= kde3_norm_integral

    return kde1_norm_vals, kde2_norm_vals, kde3_norm_vals


############################################################

def generate_cdf_and_spline(xpts, ypts):

    # Sample points
    dx = xpts[1] - xpts[0]
    #print(dx)
    #print(ypts)
    
    cdf = np.cumsum(ypts)*dx
    #print(cdf)
    
    cdf /= cdf[-1]

    spl = CubicSpline(xpts, cdf)

    return cdf, spl

############################################################

def generate_data_from_spline(spl, npts):

    #print("Generating random points..........")

    min_val,max_val = spl.x[0],spl.x[-1]
    
    start = time.time()
    #npts = 5000
    values = []
    values_random_numbers = np.random.random(npts)
    
    for i in range(npts):
        
        solutions = spl.solve(values_random_numbers[i])
        filter = (solutions>=min_val) & (solutions<=max_val)
        solutions = solutions[filter]
        
        if len(solutions)==1:
            #ynew=ynew[0]
            #print(ynew)
            values.append(solutions[0])
    
    return values



In [ ]:
#spl.x

In [ ]:
E_muon = 1600
distance = 500
e_initials_unique = df_eloss['e_initial'].unique()


de1, de2, kde1, kde2, xpts, elo, ehi = get_data_and_kde_for_interpolation(E_muon=E_muon, distance=distance, df_eloss=df_eloss, e_initials_unique=e_initials_unique, dwidth_pct=0.05)

kde1_norm_vals, kde2_norm_vals, kde3_norm_vals = morph_distributions(E_muon, kde1, kde2, xpts, elo, ehi)

cdf1, spl1 = generate_cdf_and_spline(xpts, kde1_norm_vals)
cdf2, spl2 = generate_cdf_and_spline(xpts, kde2_norm_vals)
cdf3, spl3 = generate_cdf_and_spline(xpts, kde3_norm_vals)


In [ ]:
fig,axes = plt.subplots(3,1,figsize=(12,12))
axes[0].plot(xpts, cdf1, lw=2, label=f'{elo}')
axes[0].plot(xpts, cdf2, lw=2,label=f'{ehi}')
axes[0].plot(xpts,  cdf3, lw=2, label=f'{E_muon}')
axes[0].legend()


In [ ]:
e_initials_unique

In [ ]:
######################################################################

ax = plt.figure(figsize=(8,8)).add_subplot(projection='3d')

E_muons = []
for i in range(0,10):
    
    E_muon = 10000 + 1000*i
    distance = 400

    E_muons.append(E_muon)
    
    de1, de2, kde1, kde2, xpts, elo, ehi = get_data_and_kde_for_interpolation(E_muon=E_muon, distance=distance, df_eloss=df_eloss, e_initials_unique=e_initials_unique, dwidth_pct=0.05)
    
    kde1_norm_vals, kde2_norm_vals, kde3_norm_vals = morph_distributions(E_muon, kde1, kde2, xpts, elo, ehi)
    
    y = E_muon
    z = kde3_norm_vals

    ax.fill_between(xpts, y, z,
                    xpts, y, 0,
                    #facecolors='r', 
                    alpha=.7)

    ax.set_xlabel(r'$\Delta E$ (GeV)')
    ax.set_ylabel(r'Initial $E_{\mu}$ (GeV)')


ax.set_title(f'Energy loss after {distance} meters')
####################################################

filename = f'energy_loss_for_muons_with_energy_{int(E_muons[0])}-{int(E_muons[-1])}_after_traveling_{distance}_meters.png'
plt.savefig(filename)

In [ ]:
E_muon = 1600
distance = 500
e_initials_unique = df_eloss['e_initial'].unique()


de1, de2, kde1, kde2, xpts, elo, ehi = get_data_and_kde_for_interpolation(E_muon=E_muon, distance=distance, df_eloss=df_eloss, e_initials_unique=e_initials_unique, dwidth_pct=0.05)

kde1_norm_vals, kde2_norm_vals, kde3_norm_vals = morph_distributions(E_muon, kde1, kde2, xpts, elo, ehi)

cdf1, spl1 = generate_cdf_and_spline(xpts, kde1_norm_vals)
cdf2, spl2 = generate_cdf_and_spline(xpts, kde2_norm_vals)
cdf3, spl3 = generate_cdf_and_spline(xpts, kde3_norm_vals)


npts = 500

start = time.time()
vals = generate_data_from_spline(spl3, npts)
print(f"It took {time.time()-start} seconds to generate {npts} events")

plt.hist(vals,bins=50)

;

In [ ]:
npts = 5000

start = time.time()

E_muons = np.random.normal(1600,10,npts)
distances = 500*np.random.random(npts)

vals = []

for idx in range(npts):
    
    #E_muon = E_muons[idx]
    #distance = distances[idx]
    
    #e_initials_unique = df_eloss['e_initial'].unique()
    
    #de1, de2, kde1, kde2, xpts, elo, ehi = get_data_and_kde_for_interpolation(E_muon=E_muon, distance=distance, df_eloss=df_eloss, e_initials_unique=e_initials_unique, dwidth_pct=0.05)
    
    #kde1_norm_vals, kde2_norm_vals, kde3_norm_vals = morph_distributions(E_muon, kde1, kde2, xpts, elo, ehi)
    
    #cdf1, spl1 = generate_cdf_and_spline(xpts, kde1_norm_vals)
    #cdf2, spl2 = generate_cdf_and_spline(xpts, kde2_norm_vals)
    cdf3, spl3 = generate_cdf_and_spline(xpts, kde3_norm_vals)

    val = generate_data_from_spline(spl3, 1)
    if len(val)>0:
        vals.append(val[0])

print(f"It took {time.time()-start} seconds to generate {npts} events")

plt.hist(vals,bins=50)

;

In [ ]:
# Save it
eloss_dict = {}

eloss_dict = {}

start = time.time()

nentries = 0
for E_muon in np.arange(1000,2000,1000):
    
    #E_muon = 1600
    
    eloss_dict[E_muon] = {}
    
    for distance in np.arange(100,1000,500):
    
        print(distance)
    
        #distance = 500
        e_initials_unique = df_eloss['e_initial'].unique()
        
        de1, de2, kde1, kde2, xpts, elo, ehi = get_data_and_kde_for_interpolation(E_muon=E_muon, distance=distance, df_eloss=df_eloss, e_initials_unique=e_initials_unique, dwidth_pct=0.05)
        
        kde1_norm_vals, kde2_norm_vals, kde3_norm_vals = morph_distributions(E_muon, kde1, kde2, xpts, elo, ehi)
        
        #cdf1, spl1 = generate_cdf_and_spline(xpts, kde1_norm_vals)
        #cdf2, spl2 = generate_cdf_and_spline(xpts, kde2_norm_vals)
        cdf3, spl3 = generate_cdf_and_spline(xpts, kde3_norm_vals)
        
        eloss_dict[E_muon][distance] = spl3

        nentries += 1

print(f'Time to write {nentries} entries is {time.time() - start} seconds')
start = time.time()

# Save the spline object to a file using pickle
with open('spline_object.pkl', 'wb') as file_handle:
    pickle.dump(eloss_dict, file_handle)

print(f'Time to write file is {time.time() - start} seconds')
start = time.time()

# Load the spline object from the file
with open('spline_object.pkl', 'rb') as file_handle:
    loaded_eloss_dict = pickle.load(file_handle)

print(f'Time to read file is {time.time() - start} seconds')
start = time.time()


;


In [ ]:
eloss_dict

In [ ]:
eloss_dict[1600][500]

In [ ]:
# Faster

distance = 75
# This needs to be greater than 100, since that is the lowest value we have
E_muon = 115

#e_initials = [110, 120]
e_initials_unique = df_eloss['e_initial'].unique()

#############################################

# Find window of E

# My own 
d = distance
dwidth = 0.05 * d
print(d,dwidth)

# Find the range in which we will look

#print(e_initials)
elo,ehi = find_neighbors(e_initials_unique, E_muon)
delta_e = ehi - elo

print(E_muon, delta_e, elo, ehi)

#############################################

filter_e1 = (e_initials==elo)
filter_e2 = (e_initials==ehi)
filter_d1 = (z_vals>=d-dwidth) & (z_vals<=d+dwidth)


de1 = df_eloss[filter_e1 & filter_d1]['e']
de2 = df_eloss[filter_e2 & filter_d1]['e']


#####################################

bandwidth = 0.1
kde1 = gaussian_kde(de1, bw_method=bandwidth)
kde2 = gaussian_kde(de2, bw_method=bandwidth)

#####################################
print("ranges")
#eranges = ehi_vals - elo_vals
#print(elo_vals,'\n', ehi_vals, '\n', eranges)


elo_vals, ehi_vals, elo_min,ehi_max = find_de_ranges(de1, de2)

#xpts = np.linspace(elo_min,E_muon,100)
xpts = np.linspace(elo_min,ehi_max,500)


kde1_vals = kde1.pdf(xpts)
kde2_vals = kde2.pdf(xpts)

fig,axes = plt.subplots(1,3,figsize=(12,4))
axes[0].hist(de1, range=(elo_min,ehi_max), bins=200, density=True)
axes[0].plot(xpts, kde1_vals, lw=2)
axes[0].set_xlabel(r'$\Delta$ E (GeV)')
axes[0].set_ylabel('Final energy (GeV)')
axes[0].set_title(f'Initial energy: {elo} GeV')

axes[1].hist(de2, range=(elo_min,ehi_max), bins=200, density=True)
axes[1].plot(xpts, kde2_vals, lw=2)
axes[1].set_xlabel(r'$\Delta$ E (GeV)')
axes[1].set_ylabel('Final energy (GeV)')
axes[1].set_title(f'Initial energy: {ehi} GeV')

axes[2].hist(de1, range=(elo_min,ehi_max), bins=200, density=True, label=f'E$_i$={elo} GeV')
axes[2].plot(xpts, kde1_vals, lw=2)
axes[2].hist(de2, range=(elo_min,ehi_max), bins=200, density=True, label=f'E$_i$={ehi} GeV')
axes[2].plot(xpts, kde2_vals, lw=2)
axes[2].set_xlabel(r'$\Delta$ E (GeV)')
axes[2].set_ylabel('Final energy (GeV)')
axes[2].legend()



In [ ]:
len(kde2_vals)

In [ ]:
######################################################################
# Normaize the x-scale
e_ranges = ehi_vals - elo_vals
# de1
npts = 500
xpts1 = np.linspace(elo_vals[0], ehi_vals[0], npts)
xpts2 = np.linspace(elo_vals[1], ehi_vals[1], npts)

#kde1_norm_vals = kde1.pdf(xpts1)
#kde2_norm_vals = kde2.pdf(xpts2)

kde1_norm_vals = kde1.pdf(xpts)
kde2_norm_vals = kde2.pdf(xpts)


#kde1_integral = integrate.trapezoid(kde1_norm_vals, xpts1)
#kde2_integral = integrate.trapezoid(kde2_norm_vals, xpts2)

kde1_integral = integrate.trapezoid(kde1_norm_vals, xpts)
kde2_integral = integrate.trapezoid(kde2_norm_vals, xpts)


print(kde1_integral, kde2_integral)

# Normalize the y-values so we can interpret it as probability density
# Need to consider the distances between points
#binwidth1 = xpts1[1] - xpts1[0]
#binwidth2 = xpts2[1] - xpts2[0]

kde1_norm_vals /= kde1_integral
kde2_norm_vals /= kde2_integral

scale_factor = (ehi - E_muon)/(ehi-elo)
print(f'{E_muon=}   {elo=}  {ehi=}  {scale_factor=}')
kde_between_norm_vals =  (scale_factor*kde1_norm_vals) + ((1-scale_factor)*kde2_norm_vals)


# Normalize but also take into account the bin spacing
#binwidth = xpts_new[1] - xpts_new[0]
#kde_between_norm_vals /= sum(kde_between_norm_vals)
#kde_between_norm_vals /= binwidth


#elo_new = (scale_factor*elo_vals[0]) + ((1-scale_factor)*elo_vals[1])
#ehi_new = (scale_factor*ehi_vals[0]) + ((1-scale_factor)*ehi_vals[1])

#print(f"elo_new: {elo_new}     ehi_new: {ehi_new}")
#xpts_new = np.linspace(elo_new, ehi_new, npts)

#kde_between_integral = integrate.trapezoid(kde_between_norm_vals, xpts_new)
kde_between_integral = integrate.trapezoid(kde_between_norm_vals, xpts)
kde_between_norm_vals /= kde_between_integral



xpts_norm = np.linspace(0,1, npts)

fig,axes = plt.subplots(3,1,figsize=(12,12))
axes[0].plot(xpts, kde1_norm_vals, lw=2, label=f'{elo}')
axes[0].plot(xpts, kde2_norm_vals, lw=2,label=f'{ehi}')
axes[0].legend()

#axes[1].plot(xpts_norm, kde1_norm_vals, lw=2, label=f'{elo}')
#axes[1].plot(xpts_norm, kde2_norm_vals, lw=2, label=f'{ehi}')
#axes[1].plot(xpts_norm, kde_between_norm_vals, lw=2, label=f'{E_muon}')
#axes[1].legend()

axes[1].plot(xpts, kde1_norm_vals, lw=2, label=f'{elo}')
axes[1].plot(xpts, kde2_norm_vals, lw=2,label=f'{ehi}')
axes[1].plot(xpts, kde_between_norm_vals, lw=2, label=f'{E_muon}')
axes[1].legend()


print(sum(kde1_norm_vals))
print(sum(kde2_norm_vals))
print(sum(kde_between_norm_vals))

In [ ]:
def generate_cdf(xpts, ypts):

    # Sample points
    dx = xpts[1] - xpts[0]
    #print(dx)
    #print(ypts)
    
    cdf = np.cumsum(ypts)*dx
    #print(cdf)
    
    cdf /= cdf[-1]

    return cdf

cdf1 = generate_cdf(xpts, kde1_norm_vals)
cdf2 = generate_cdf(xpts, kde2_norm_vals)
cdf3 = generate_cdf(xpts, kde_between_norm_vals)


print("Making the spline........")

spl = CubicSpline(xpts, cdf3)


####
#spl = CubicSpline(xpts_new, cdf_between)

#new_vals = spl.solve(new_vals_rand)

#print(new_vals_rand)
#print()
#enew_min = min(xpts_new)
#enew_max = max(xpts_new)

print("Generating random points..........")

start = time.time()
new_npts = 5000
ynew_pts = []
new_vals_rand = np.random.random(new_npts)

for i in range(new_npts):
    
    ynew = spl.solve(new_vals_rand[i])
    filter = (ynew>enew_min) & (ynew<enew_max)
    ynew = ynew[filter]
    if len(ynew)==1:
        ynew=ynew[0]
        #print(ynew)
        ynew_pts.append(ynew)

print(f'Generated {new_npts} points in {time.time() - start} seconds')


In [ ]:
kde_intermediate = gaussian_kde(kde_between_norm_vals)

ynew_pts2 = kde_intermediate.resample(10000)
#print(ynew_pts2[0])
plt.hist(ynew_pts2[0],bins=50)

fig,axes = plt.subplots(3,1,figsize=(12,12))
axes[0].plot(xpts, cdf1, lw=2, label=f'{elo}')
axes[0].plot(xpts, cdf2, lw=2,label=f'{ehi}')
axes[0].plot(xpts,  cdf3, lw=2, label=f'{E_muon}')
axes[0].legend()

#axes[1].hist(de1,     range=(elo_min,ehi_max), bins=200, density=True, alpha=0.4, label=f'{elo}')
axes[1].hist(de2,     range=(elo_min,ehi_max), bins=200, density=True, alpha=0.4, label=f'{ehi}')
axes[1].hist(ynew_pts,range=(elo_min,ehi_max), bins=200, density=True, alpha=0.75, label=f'{E_muon}')
axes[1].legend()

axes[2].hist(de1,     range=(elo_min,ehi_max), bins=200, density=True, alpha=0.4, label=f'{elo}')
#axes[1].hist(de2,     range=(elo_min,ehi_max), bins=200, density=True, alpha=0.4, label=f'{ehi}')
axes[2].hist(ynew_pts,range=(elo_min,ehi_max), bins=200, density=True, alpha=0.4, label=f'{E_muon}')
axes[2].legend()


#axes[2].hist(de1, range=(elo_min,ehi_max), bins=200, density=True, alpha=0.4, label=f'{elo}')
#aes[2].hist(de1, range=(elo_min,ehi_max), bins=200, density=True, alpha=0.4, label=f'{ehi}')
#axes[2].hist(ynew_pts,range=(elo_min,ehi_max), bins=200, density=True, alpha=0.75, label=f'{E_muon}')
#axes[2].legend()

;


In [ ]:
spl.solve(0.5, extrapolate='periodic')

In [ ]:
######################################################################

ax = plt.figure().add_subplot(projection='3d')


for i in range(0,2):
    if i==0:
        y = elo
        z = kde1_vals
    else:
        y = ehi
        z = kde2_vals
    ax.fill_between(xpts, y, z,
                    xpts, y, 0,
                    facecolors='r', alpha=.7)


####################################################

# Frequencies


#e_range = ehi_max[i] - elo_min[i]
#xpts -= elo_min[i]
#xpts /= e_range


In [ ]:
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt

# 1. Define the x-axis for plotting
x = np.linspace(-5, 10, 500)

# 2. Define the two initial distributions (e.g., two normal distributions)
dist1 = norm(loc=0, scale=1)   # Mean=0, StdDev=1
dist2 = norm(loc=5, scale=2)   # Mean=5, StdDev=2

pdf1 = dist1.pdf(x)
pdf2 = dist2.pdf(x)

# 3. Create morphed distributions by varying alpha
alphas = np.linspace(0, 1, 5)  # Weights for the morphing

plt.figure(figsize=(10, 6))
plt.plot(x, pdf1, label='Distribution 1 (α=0)', linestyle='--', color='blue')
plt.plot(x, pdf2, label='Distribution 2 (α=1)', linestyle='--', color='red')

for alpha in alphas:
    if alpha == 0 or alpha == 1:
        continue
    # Linearly interpolate the PDFs
    morphed_pdf = (1 - alpha) * pdf1 + alpha * pdf2
    plt.plot(x, morphed_pdf, label=f'Morphed (α={alpha:.1f})')

plt.title('Morphing Two Distributions with PDF Interpolation')
plt.xlabel('x')
plt.ylabel('Probability Density')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
import numpy as np
from scipy.stats import norm
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt

# 1. Define initial distributions and sample from them
np.random.seed(42)
samples1 = norm.rvs(loc=0, scale=1, size=1000)
samples2 = norm.rvs(loc=5, scale=2, size=1000)

# 2. Compute empirical CDFs for each set of samples
ecdf1_x = np.sort(samples1)
ecdf1_y = np.linspace(0, 1, len(samples1))
ecdf2_x = np.sort(samples2)
ecdf2_y = np.linspace(0, 1, len(samples2))

# 3. Interpolate quantiles (inverse CDF)
quantiles_y = np.linspace(0, 1, 500)
interp_q1 = interp1d(ecdf1_y, ecdf1_x, bounds_error=False, fill_value=(ecdf1_x[0], ecdf1_x[-1]))
interp_q2 = interp1d(ecdf2_y, ecdf2_x, bounds_error=False, fill_value=(ecdf2_x[0], ecdf2_x[-1]))

# 4. Morph and visualize
alphas = [0, 0.25, 0.5, 0.75, 1]

plt.figure(figsize=(10, 6))

for alpha in alphas:
    # Linearly interpolate quantiles
    morphed_quantiles_x = (1 - alpha) * interp_q1(quantiles_y) + alpha * interp_q2(quantiles_y)
    
    # Plot the resulting distribution as a histogram
    plt.hist(morphed_quantiles_x, bins=50, density=True, histtype='step', 
             label=f'Morphed (α={alpha:.2f})', alpha=0.8, range=(-5, 10))

plt.title('Morphing Two Distributions with Quantile Interpolation')
plt.xlabel('x')
plt.ylabel('Density')
plt.legend()
plt.grid(True)
plt.show()


https://scikit-learn.org/stable/auto_examples/neighbors/plot_kde_1d.html#sphx-glr-auto-examples-neighbors-plot-kde-1d-py

In [ ]:
# Example

import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm

from sklearn.neighbors import KernelDensity

# ----------------------------------------------------------------------
# Plot the progression of histograms to kernels
np.random.seed(1)
N = 20
X = np.concatenate(
    (np.random.normal(0, 1, int(0.3 * N)), np.random.normal(5, 1, int(0.7 * N)))
)[:, np.newaxis]
X_plot = np.linspace(-5, 10, 1000)[:, np.newaxis]
bins = np.linspace(-5, 10, 10)

fig, ax = plt.subplots(2, 2, sharex=True, sharey=True)
fig.subplots_adjust(hspace=0.05, wspace=0.05)

# histogram 1
ax[0, 0].hist(X[:, 0], bins=bins, fc="#AAAAFF", density=True)
ax[0, 0].text(-3.5, 0.31, "Histogram")

# histogram 2
ax[0, 1].hist(X[:, 0], bins=bins + 0.75, fc="#AAAAFF", density=True)
ax[0, 1].text(-3.5, 0.31, "Histogram, bins shifted")

# tophat KDE
kde = KernelDensity(kernel="tophat", bandwidth=0.75).fit(X)
log_dens = kde.score_samples(X_plot)
ax[1, 0].fill(X_plot[:, 0], np.exp(log_dens), fc="#AAAAFF")
ax[1, 0].text(-3.5, 0.31, "Tophat Kernel Density")

# Gaussian KDE
kde = KernelDensity(kernel="gaussian", bandwidth=0.75).fit(X)
log_dens = kde.score_samples(X_plot)
ax[1, 1].fill(X_plot[:, 0], np.exp(log_dens), fc="#AAAAFF")
ax[1, 1].text(-3.5, 0.31, "Gaussian Kernel Density")

for axi in ax.ravel():
    axi.plot(X[:, 0], np.full(X.shape[0], -0.01), "+k")
    axi.set_xlim(-4, 9)
    axi.set_ylim(-0.02, 0.34)

for axi in ax[:, 0]:
    axi.set_ylabel("Normalized Density")

for axi in ax[1, :]:
    axi.set_xlabel("x")

# ----------------------------------------------------------------------
# Plot all available kernels
X_plot = np.linspace(-6, 6, 1000)[:, None]
X_src = np.zeros((1, 1))

fig, ax = plt.subplots(2, 3, sharex=True, sharey=True)
fig.subplots_adjust(left=0.05, right=0.95, hspace=0.05, wspace=0.05)


def format_func(x, loc):
    if x == 0:
        return "0"
    elif x == 1:
        return "h"
    elif x == -1:
        return "-h"
    else:
        return "%ih" % x


for i, kernel in enumerate(
    ["gaussian", "tophat", "epanechnikov", "exponential", "linear", "cosine"]
):
    axi = ax.ravel()[i]
    log_dens = KernelDensity(kernel=kernel).fit(X_src).score_samples(X_plot)
    axi.fill(X_plot[:, 0], np.exp(log_dens), "-k", fc="#AAAAFF")
    axi.text(-2.6, 0.95, kernel)

    axi.xaxis.set_major_formatter(plt.FuncFormatter(format_func))
    axi.xaxis.set_major_locator(plt.MultipleLocator(1))
    axi.yaxis.set_major_locator(plt.NullLocator())

    axi.set_ylim(0, 1.05)
    axi.set_xlim(-2.9, 2.9)

ax[0, 1].set_title("Available Kernels")

# ----------------------------------------------------------------------
# Plot a 1D density example
N = 100
np.random.seed(1)
X = np.concatenate(
    (np.random.normal(0, 1, int(0.3 * N)), np.random.normal(5, 1, int(0.7 * N)))
)[:, np.newaxis]

X_plot = np.linspace(-5, 10, 1000)[:, np.newaxis]

true_dens = 0.3 * norm(0, 1).pdf(X_plot[:, 0]) + 0.7 * norm(5, 1).pdf(X_plot[:, 0])

fig, ax = plt.subplots()
ax.fill(X_plot[:, 0], true_dens, fc="black", alpha=0.2, label="input distribution")
colors = ["navy", "cornflowerblue", "darkorange"]
kernels = ["gaussian", "tophat", "epanechnikov"]
lw = 2

for color, kernel in zip(colors, kernels):
    kde = KernelDensity(kernel=kernel, bandwidth=0.5).fit(X)
    log_dens = kde.score_samples(X_plot)
    ax.plot(
        X_plot[:, 0],
        np.exp(log_dens),
        color=color,
        lw=lw,
        linestyle="-",
        label="kernel = '{0}'".format(kernel),
    )

ax.text(6, 0.38, "N={0} points".format(N))

ax.legend(loc="upper left")
ax.plot(X[:, 0], -0.005 - 0.01 * np.random.random(X.shape[0]), "+k")

ax.set_xlim(-4, 9)
ax.set_ylim(-0.02, 0.4)
plt.show()

In [ ]:
#X
X_plot